In [4]:
!pip install salesforce-lavis

  Using cached salesforce_lavis-1.0.2-py3-none-any.whl (1.8 MB)
You should consider upgrading via the '/gpfs/home2/scur2863/ondemand/.venv/bin/python -m pip install --upgrade pip' command.


In [1]:
from PIL import Image
from lavis.models import load_model_and_preprocess
import torch
import json
import os

/home/scur2863/ondemand/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scur2863/ondemand/.venv/lib64/python3.9/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/scur2863/ondemand/.venv/lib64/python3.9/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [2]:
from lavis.models import model_zoo
print(model_zoo)

Architectures                  Types
albef_classification           ve
albef_feature_extractor        base
albef_nlvr                     nlvr
albef_pretrain                 base
albef_retrieval                coco, flickr
albef_vqa                      vqav2
alpro_qa                       msrvtt, msvd
alpro_retrieval                msrvtt, didemo
blip_caption                   base_coco, large_coco
blip_classification            base
blip_feature_extractor         base
blip_image_text_matching       base, large
blip_nlvr                      nlvr
blip_pretrain                  base
blip_retrieval                 coco, flickr
blip_vqa                       vqav2, okvqa, aokvqa
blip2_opt                      pretrain_opt2.7b, pretrain_opt6.7b, caption_coco_opt2.7b, caption_coco_opt6.7b
blip2_t5                       pretrain_flant5xl, pretrain_flant5xl_vitL, pretrain_flant5xxl, caption_coco_flant5xl
blip2_feature_extractor        pretrain, pretrain_vitL, coco
blip2                      

The TextCaps dataset can be downloaded from https://textvqa.org/textcaps/

In [7]:
# Load model and preprocessors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, vis_processors, txt_processors = load_model_and_preprocess( name="albef_feature_extractor", model_type="base", is_eval=True, device=device)

# Directory containing images
image_dir = "dataset/"

# Path to JSON file containing captions
caption_json_path = "TextCaps_0.1_train.json"

# Batch size for processing
batch_size = 16

# Load captions JSON
with open(caption_json_path, "r") as f:
    caption_data = json.load(f)

# Assert the structure of the JSON file
assert "data" in caption_data, "'data' key not found in JSON"
assert isinstance(caption_data["data"], list), "'data' key should contain a list of image dictionaries"

# Parse the captions and image IDs
image_captions = {}
for item in caption_data["data"]:
    assert "image_id" in item, "'image_id' key not found in one of the image dictionaries"
    #assert "reference_strs" in item, "'reference_strs' key not found in one of the image dictionaries"
    #assert isinstance(item["reference_strs"], list) and len(item["reference_strs"]) == 5, \
       # "'reference_strs' must be a list containing exactly 5 captions"

    image_id = item["image_id"]
    captions = item["caption_str"]
    
    image_captions[image_id] = captions

## Image Embeddings

In [ ]:
import pandas as pd

def dump_parquet(images_ids,image_embeddings):
  image_embeddings = torch.cat(image_embeddings, dim=0).cpu()
  embeddings_list = image_embeddings.tolist()

  df_ = pd.DataFrame({
    "id": images_ids,

    "embedding": embeddings_list})
  output_path = 'image_embeddings_albef_new2.parquet'

  # Save as Parquet
  df_.to_parquet(output_path, engine='pyarrow')
  print(f"Embeddings successfully saved to {output_path}")

In [6]:
# Helper function to process a batch of images and captions
batch_size = 4
import numpy as np
np.Inf = np.inf
def process_batch(image_paths):
    processed_images = []
    #processed_texts = []

    for path in image_paths:
        raw_image = Image.open(path).convert("RGB")  # Ensure all images are RGB
        processed_images.append(vis_processors["eval"](raw_image).unsqueeze(0))  # Process image
        #processed_texts.append([txt_processors["eval"](caption) for caption in captions])  # Process each caption

    # Stack processed images into a single tensor
    batched_images = torch.cat(processed_images, dim=0).to(device)
    
    textt=[]
    for k in range(len(batched_images)):
        textt.append('rr r rr')
    
    # Flatten captions into a single list for this batch
    #flat_texts = [item for sublist in processed_texts for item in sublist]

    # Create a sample for the model
    sample = {"image_id":torch.tensor([k+1 for k in range(len(batched_images))]).to(device),"image": batched_images, "text_input": textt, "epoch": 0, "iters": 0, "num_iters_per_epoch": 100}
    return sample

# Get all image paths and verify matching captions
all_image_paths = []
#all_captions = []

for image_id, captions in image_captions.items():
    image_path = os.path.join(image_dir, image_id +'.jpg')
    assert os.path.exists(image_path), f"Image file not found: {image_path}"
    all_image_paths.append(image_path)
    #all_captions.append(captions)

# Process images and captions in batches
image_embeddings = []
image_ids=[]
#text_embeddings = []
#similarity_matrices = []

for i in range(19000, len(all_image_paths)-1, batch_size):
    # Get batch of image paths and captions
    batch_paths = all_image_paths[i:i+batch_size]
    #batch_captions = all_captions[i:i+batch_size]

    # Prepare the sample
    sample = process_batch(batch_paths)
    

    # Extract features
    with torch.no_grad():
        output = model(sample)
    features_text = model.vision_proj(output.intermediate_output.image_embeds[:,0,:])
    # Append embeddings
    image_embeddings.append(features_text)
    
    #text_embeddings.append(features_text.text_embeds_proj)
    image_ids.extend(list(image_captions.keys())[i:i+batch_size])
    
    # Compute similarity between image and captions (averaging over captions per image)
    '''
    for idx in range(len(batch_paths)):
        image_feature = features_image.image_embeds_proj[idx, 0, :]
        text_features = features_text.text_embeds_proj[idx*5:(idx+1)*5, 0, :]  # 5 captions per image
        similarity = torch.mean(image_feature @ text_features.t())
        similarity_matrices.append(similarity)
      '''
    if i%1000==0:
        print(i)
        dump_parquet(image_ids,image_embeddings)


# Stack all batches together
dump_parquet(image_ids,image_embeddings)
image_embeddings = torch.cat(image_embeddings, dim=0)  # [Total Images, Seq Length, Embedding Dim]
#text_embeddings = torch.cat(text_embeddings, dim=0)    # [Total Captions, Seq Length, Embedding Dim]

# Final similarity matrix (aggregated)
#similarity_matrix = torch.stack(similarity_matrices)

# Output results
print("Total Images Processed:", len(all_image_paths))
print("Image Embedding Shape:", image_embeddings.shape)
#print("Text Embedding Shape:", text_embeddings.shape)
#print("Similarity Matrix Shape:", similarity_matrix.shape)

19000
Embeddings successfully saved to image_embeddings_albef_new2.parquet
20000
Embeddings successfully saved to image_embeddings_albef_new2.parquet
21000
Embeddings successfully saved to image_embeddings_albef_new2.parquet
Embeddings successfully saved to image_embeddings_albef_new2.parquet
Total Images Processed: 21953
Image Embedding Shape: torch.Size([2952, 256])


## Text Embeddings

In [4]:
import pandas as pd

def dump_parquet_text(senten_ids,text_embeddings):
  text_embeddings = torch.cat(text_embeddings, dim=0).to(device)
  embeddings_list = text_embeddings.tolist()
  df_ = pd.DataFrame({
    "id": senten_ids,
    "embedding": embeddings_list}
                     )
  output_path = 'text_embeddings_albef_new2.parquet'
  #print(df_)
  # Save as Parquet
  df_.to_parquet(output_path, engine='pyarrow')
  print(f"Embeddings successfully saved to {output_path}")

In [7]:
import numpy as np
np.Inf = np.inf
def process_batch_text(captions_list):
    processed_images = []
    processed_texts = []

    for caption in  captions_list:
        
        #raw_image = Image.open(path).convert("RGB")  # Ensure all images are RGB
        #processed_images.append(vis_processors["eval"](raw_image).unsqueeze(0))  # Process image
        processed_texts.append(txt_processors["eval"](caption))  # Process each caption
        #processed_images.append(torch.rand(16,224,224))
    # Stack processed images into a single tensor
    
    batched_images = torch.cat([torch.zeros(1,3,384,384)]*len(processed_texts), dim=0).to(device)
    #batched_text = [item for sublist in processed_texts]
    # Flatten captions into a single list for this batch
    assert len(processed_texts) == len(batched_images) , f"mismatch{len(processed_texts)}!={len(batched_images) }"
     #[item for sublist in processed_texts]
    #batched_images=torch.zeros(5,3,224,224)
    # Create a sample for the model
    sample = {"image_id":torch.tensor([k+1 for k in range(len(batched_images))]).to(device),"image": batched_images, "text_input": processed_texts, "epoch": 0, "iters": 0, "num_iters_per_epoch": 100}
    return sample

# Get all image paths and verify matching captions
all_image_paths = []
all_captions = []

# Parse the captions and image IDs
image_captions_full = {}
for item in caption_data["data"]:
    caption_id = item["caption_id"]
    captions = item["caption_str"]
    image = item["image_id"]
    image_captions_full[caption_id] = {'text':captions, 'image':image}


for item in image_captions_full.values():
    image_id =item['image']
    caption = item['text']
    #image_path = os.path.join(image_dir, image_id +'.jpg')
    #assert os.path.exists(image_path), f"Image file not found: {image_path}"
    #all_image_paths.append(image_path)
    all_captions.append(caption)

# Process images and captions in batches
image_id = []
text_embedding = []
sentence_id=[]

for i in range(, len(all_captions), batch_size):

    # Get batch of image paths and captions
    batch_captions = all_captions[i:i+batch_size]
    #print(batch_captions)

    # Prepare the sample
    sample = process_batch_text(batch_captions)

    assert not torch.isnan(sample["image"]).any()
    assert not torch.isinf(sample["image"]).any()
    # Extract features
    #features_text = model2.extract_features(sample, mode="text")
    with torch.no_grad():
        output = model(sample)
    
    features_text = model.text_proj(output.intermediate_output.text_embeds[:,0,:])
    # Append embeddings
    text_embedding.append(features_text)
    
    sentence_id.extend(list(image_captions_full.keys())[i:i+batch_size])
    
    # Compute similarity between image and captions (averaging over captions per image)
    '''
    for idx in range(len(batch_paths)):
        image_feature = features_image.image_embeds_proj[idx, 0, :]
        text_features = features_text.text_embeds_proj[idx*5:(idx+1)*5, 0, :]  # 5 captions per image
        similarity = torch.mean(image_feature @ text_features.t())
        similarity_matrices.append(similarity)
      '''
    if i%5001==0:
      print(i)
      dump_parquet_text(sentence_id,text_embedding)


# Stack all batches together
dump_parquet_text(sentence_id,text_embedding)
text_embeddings = torch.cat(text_embeddings, dim=0)    # [Total Captions, Seq Length, Embedding Dim]

# Final similarity matrix (aggregated)
#similarity_matrix = torch.stack(similarity_matrices)

# Output results
#print("Total Images Processed:", len(all_image_paths))
print("Text Embedding Shape:", text_embeddings.shape)


../aten/src/ATen/native/cuda/TensorCompare.cu:110: _assert_async_cuda_kernel: block: [0,0,0], thread: [0,0,0] Assertion `invalid multinomial distribution (sum of probabilities <= 0)` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [114]:
batch_size=4
import numpy as np
np.Inf = np.inf
def process_batch_text(captions_list):
    processed_images = []
    processed_texts = []

    for caption in  captions_list:
        
        #raw_image = Image.open(path).convert("RGB")  # Ensure all images are RGB
        #processed_images.append(vis_processors["eval"](raw_image).unsqueeze(0))  # Process image
        processed_texts.append(txt_processors["eval"](caption))  # Process each caption
        #processed_images.append(torch.rand(16,224,224))
    # Stack processed images into a single tensor
    
    batched_images = torch.cat([torch.zeros(1,3,224,224)]*len(processed_texts), dim=0).to(device)
    #batched_text = [item for sublist in processed_texts]
    # Flatten captions into a single list for this batch
    assert len(processed_texts) == len(batched_images) , f"mismatch{len(processed_texts)}!={len(batched_images) }"
     #[item for sublist in processed_texts]
    #batched_images=torch.zeros(5,3,224,224)
    # Create a sample for the model
    sample = {"image": torch.zeros(1,3,224,224), "text_input": ['A black and white graphic drawing without text on it'], "epoch": 0, "iters": 0, "num_iters_per_epoch": 100}
    return sample

# Get all image paths and verify matching captions
all_image_paths = []
all_captions = []


for item in image_captions_full.values():
    image_id =item['image']
    caption = item['text']
    #image_path = os.path.join(image_dir, image_id +'.jpg')
    #assert os.path.exists(image_path), f"Image file not found: {image_path}"
    #all_image_paths.append(image_path)
    all_captions.append((caption)

# Process images and captions in batches
image_id = []
text_embedding = []
sentence_id=[]
for i in range(1):

    # Get batch of image paths and captions
    batch_captions = ['100109764']
    #print(batch_captions)

    # Prepare the sample
    sample = process_batch_text(batch_captions)

    assert not torch.isnan(sample["image"]).any()
    assert not torch.isinf(sample["image"]).any()
    # Extract features
    #features_text = model2.extract_features(sample, mode="text")
    with torch.no_grad():
        output = model2(sample)
    
    features_text = model2.text_proj(output.intermediate_output.text_embeds[:,0,:])
    # Append embeddings
    text_embedding.append(features_text)
    
    sentence_id.extend(list(image_captions_full.keys())[i:i+batch_size])
    
    # Compute similarity between image and captions (averaging over captions per image)
    '''
    for idx in range(len(batch_paths)):
        image_feature = features_image.image_embeds_proj[idx, 0, :]
        text_features = features_text.text_embeds_proj[idx*5:(idx+1)*5, 0, :]  # 5 captions per image
        similarity = torch.mean(image_feature @ text_features.t())
        similarity_matrices.append(similarity)
      '''
    if i%5001==0:
      print(i)
      dump_parquet_text(sentence_id,text_embedding)


# Stack all batches together
dump_parquet_text(sentence_id,text_embedding)
text_embeddings = torch.cat(text_embeddings, dim=0)    # [Total Captions, Seq Length, Embedding Dim]

# Final similarity matrix (aggregated)
#similarity_matrix = torch.stack(similarity_matrices)

# Output results
#print("Total Images Processed:", len(all_image_paths))
print("Text Embedding Shape:", text_embeddings.shape)
#print("Text Embedding Shape:", text_embeddings.shape)
#print("Similarity Matrix Shape:", similarity_matrix.shape)

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

## Metadata train/val/test split

In [ ]:
import json
import random


# Path to JSON file containing captions
caption_json_path = "TextCaps_0.1_train.json"

# Batch size for processing
batch_size = 16

# Load captions JSON
with open(caption_json_path, "r") as f:
    data = json.load(f)
# Extract unique picture IDs
unique_picture_ids = {item["image_id"] for item in data['data']}

# Shuffle unique picture IDs
unique_picture_ids = list(unique_picture_ids)
random.shuffle(unique_picture_ids)

# Split picture IDs into train, val, and test
num_pictures = len(unique_picture_ids)
train_split = int(0.6 * num_pictures)
val_split = int(0.2 * num_pictures)

train_ids = set(unique_picture_ids[:train_split])
val_ids = set(unique_picture_ids[train_split:train_split + val_split])
test_ids = set(unique_picture_ids[train_split + val_split:])

# Add 'type' label to each item in the data
for item in data['data']:
    if item["image_id"] in train_ids:
        item["type"] = "train"
    elif item["image_id"] in val_ids:
        item["type"] = "val"
    elif item["image_id"] in test_ids:
        item["type"] = "test"

# Save updated JSON

with open("metadata_split.json", "w") as f:
    json.dump(data, f, separators=(",", ":"), ensure_ascii=False)
print("Data split into train, val, and test sets.")


## Similarity

In [1]:
import datasets
from datasets import load_dataset

/home/scur2863/ondemand/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
import pandas as pd
output_path = 'image_embeddings_albef_new.parquet'

image_df = pd.read_parquet(output_path, engine='pyarrow')


In [57]:
output_path = 'text_embeddings_albef_new.parquet'
text_df = pd.read_parquet(output_path, engine='pyarrow')

,id,embedding
0,100000000,"[-0.3620331287384033, -0.16910064220428467, -1..."
1,100000001,"[0.4743715822696686, 1.591766595840454, -1.348..."
2,100000002,"[0.14031903445720673, 0.3389507234096527, -1.4..."
3,100000003,"[-0.036332644522190094, 0.2871374189853668, -1..."
4,100000004,"[0.053886838257312775, 0.24197791516780853, -1..."
...,...,...
109760,100109760,"[0.629140317440033, 0.41149383783340454, 0.514..."
109761,100109761,"[0.629140317440033, 0.41149383783340454, 0.514..."
109762,100109762,"[0.629140317440033, 0.41149383783340454, 0.514..."
109763,100109763,"[0.629140317440033, 0.41149383783340454, 0.514..."


In [58]:
# Parse the captions and image IDs
image_captions_full = {}
for item in caption_data["data"]:
    caption_id = item["caption_id"]
    captions = item["caption_str"]
    image = item["image_id"]
    image_captions_full[caption_id] = {'text':caption_id, 'image':image}

In [53]:
import numpy as np

In [60]:
similarities=[]
for item in list(image_captions_full.values()):
    
    image_id =item['image']
    caption = item['text']
    
    text_emb=torch.tensor(text_df[text_df['id']==caption]['embedding'].values[0])
    
    
    image_emb=torch.tensor(image_df[image_df['id']==image_id]['embedding'].values[0])
    #print(np.shape(image_emb.reshape(( 1,256))))
    # Normalize embeddings to unit vectors
    text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)
    image_emb = image_emb / image_emb.norm(dim=-1, keepdim=True)
    
    # Compute cosine similarity
    similarity = image_emb @ text_emb.t()
    similarities.append(similarity.item())
    


In [61]:
np.mean(similarities)

np.float64(0.0915124160735395)

In [8]:
dense_embs = load_dataset('alvarodelamaza/textcaps-blip-dense', data_files={"img_emb": "image_embeddings_blip.parquet",
                                                        "text_emb": "text_embeddings_blip.parquet"}, keep_in_memory=True).with_format("numpy")
text_ids = dense_embs['text_emb']["id"]
text_embs = dense_embs['text_emb']['emb']
img_ids = dense_embs['img_emb']['id']
img_embs = dense_embs['img_emb']['emb']

NameError: name 'hf_hub_download' is not defined

In [44]:
len(text_df[text_df['id']==caption]['embedding'][0]

KeyError: 0